In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import sys, os
import numpy as np
import scipy 
import tensorflow as tf

import imageio
import gzip
from PIL import Image

import seaborn as sns
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
#from tqdm import trange, tqdm

import keras
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Optimizer
from keras.callbacks import Callback
from collections import OrderedDict

from helpers import utils
# from helpers import protocols
# from helpers.keras_utils import LossHistory
# from helpers.optimizers import KOOptimizer

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

EXTRACT LABELS

In [44]:
# extracrt labels from local file
train_labels = extract_labels("MNIST-data/train-labels-idx1-ubyte.gz", 60000)
eval_labels = extract_labels("MNIST-data/t10k-labels-idx1-ubyte.gz", 10000)
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(eval_labels, num_classes)

print(np.shape(train_labels))
print(np.shape(eval_labels))
print(np.shape(y_train))
print(np.shape(y_test))

(60000,)
(10000,)
(60000, 10)
(10000, 10)


CONSTRUCT DATASETS


In [22]:
# create local dataset based off of permuted MNIST data
def createDataset(name, trainsrc, testsrc):
	print("Beginning import:", name)
	train = np.zeros((train_size, img_rows, img_cols), dtype=np.float32)
	test = np.zeros((test_size, img_rows, img_cols), dtype=np.float32)

	imgstrain = ["MNIST-processed-training/{0}{1}.png".format(trainsrc, k) for k in range(1, train_size)]
	imgstest = ["MNIST-processed-test/{0}{1}.png".format(testsrc, k) for k in range(1, test_size + 1)]

	for i in range(len(imgstrain)):
		img = np.array(Image.open(imgstrain[i]))
		train[i, :, :] = img

	for  i in range(len(imgstest)):
		img = np.array(Image.open(imgstest[i]))
		test[i, :, :] = img

	print("Completed import:", name)

	return (train, test)

PARAMETERS

In [13]:
# # Data params
# input_dim = 784
# output_dim = 10

# input image dimensions
img_rows, img_cols = 28, 28

# # Network params
# n_hidden_units = 2000
# activation_fn = tf.nn.relu

# Optimization params
batch_size = 256
num_classes = 10
epochs = 5 # epochs per task
# learning_rate=1e-3
# xi = 0.1

# Reset optimizer after each age
# reset_optimizer = False

In [39]:
# Optimization parameters
img_rows, img_cols = 28, 28
train_size = 60000
test_size = 10000
batch_size = 256
num_classes = 10
epochs = 5
lr = 0.01
xi = 0.1

# Architecture params
hidden_neurons = 3000
activation_fn = tf.nn.relu 
output_fn = tf.nn.softmax

# data
input_size = 784
output_size = 10

In [40]:
# import all permuted datasets (paths = local paths in filesystem)
def importData():
    srcs = [("original", "original/original", "original/test-original"),\
		("rot90", "rot90/rot90", "rot90/test-rot90"), \
		("fliplr", "fliplr/fliplr", "fliplr/test-fliplr"), \
		("flipud", "flipud/flipud", "flipud/test-flipud"), \
		("check", "checkerboard/fullcheck", "checkerboard/test-checkerboard"), \
		("inv", "Inv/inv", "inv/test-inv"), \
		("cutud","cutud/cutUD", "cutud/test-cutud"),\
		("invbot", "invbot/invbot", "invbot/test-invbot"), \
		]
    
    datasets = list(map(lambda x: createDataset(x[0], x[1], x[2]), srcs))

    data = dict()
    
    for i in range(len(datasets)):
        x_train = datasets[i][0]
        x_test = datasets[i][1]
		#if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
        x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)
        input_shape = (1, img_rows * img_cols)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        data[i] = {"train": x_train, "test": x_test}

    return data

In [41]:

data = importData()

Beginning import: original
Completed import: original
Beginning import: rot90
Completed import: rot90
Beginning import: fliplr
Completed import: fliplr
Beginning import: flipud
Completed import: flipud
Beginning import: check
Completed import: check
Beginning import: inv
Completed import: inv
Beginning import: cutud
Completed import: cutud
Beginning import: invbot
Completed import: invbot


CONSTRUCT MODEL

In [42]:
model = Sequential()
model.add(Dense(hidden_neurons, activation=activation_fn, input_dim=input_size))
model.add(Dense(hidden_neurons, activation=activation_fn))
model.add(Dense(output_size, activation=output_fn))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [58]:
modelx = Sequential()
modelx.add(Dense(hidden_neurons, activation=activation_fn, input_dim=input_size))
modelx.add(Dense(hidden_neurons, activation=activation_fn))
modelx.add(Dense(output_size, activation=output_fn))
modelx.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

modelx.fit(data[1]["train"], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(data[1]["test"], y_test))
score = modelx.evaluate(data[1]["test"], y_test, verbose=0)
print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 98s 2ms/step - loss: 0.3014 - acc: 0.9071 - val_loss: 0.2053 - val_acc: 0.9308
Epoch 2/5
60000/60000 [==============================] - 98s 2ms/step - loss: 0.0983 - acc: 0.9696 - val_loss: 0.1261 - val_acc: 0.9622
Epoch 3/5
60000/60000 [==============================] - 114s 2ms/step - loss: 0.0638 - acc: 0.9803 - val_loss: 0.0629 - val_acc: 0.9794
Epoch 4/5
60000/60000 [==============================] - 115s 2ms/step - loss: 0.0434 - acc: 0.9865 - val_loss: 0.0645 - val_acc: 0.9803
Epoch 5/5
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0289 - acc: 0.9913 - val_loss: 0.0570 - val_acc: 0.9821
Test accuracy: 0.9821


In [47]:
model.fit(data[0]["train"], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(data[0]["test"], y_test))
score = model.evaluate(data[0]["test"], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save trained model
model.save('model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 197s 3ms/step - loss: 1.5657 - acc: 0.5134 - val_loss: 1.0039 - val_acc: 0.6727
Epoch 2/5
60000/60000 [==============================] - 196s 3ms/step - loss: 0.5083 - acc: 0.8320 - val_loss: 0.2934 - val_acc: 0.9063
Epoch 3/5
60000/60000 [==============================] - 197s 3ms/step - loss: 0.3421 - acc: 0.8905 - val_loss: 0.5574 - val_acc: 0.8176
Epoch 4/5
60000/60000 [==============================] - 188s 3ms/step - loss: 0.2703 - acc: 0.9130 - val_loss: 0.2215 - val_acc: 0.9290
Epoch 5/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.2314 - acc: 0.9267 - val_loss: 0.2435 - val_acc: 0.9221
Test loss: 0.24352554704993964
Test accuracy: 0.9221


In [48]:
# Reload model
model2 = load_model('model.h5')

model2.fit(data[1]["train"], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(data[1]["test"], y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 183s 3ms/step - loss: 0.3134 - acc: 0.9108 - val_loss: 0.1967 - val_acc: 0.9326
Epoch 2/5
60000/60000 [==============================] - 196s 3ms/step - loss: 0.1040 - acc: 0.9681 - val_loss: 0.1114 - val_acc: 0.9652
Epoch 3/5
60000/60000 [==============================] - 262s 4ms/step - loss: 0.0654 - acc: 0.9800 - val_loss: 0.0797 - val_acc: 0.9744
Epoch 4/5
60000/60000 [==============================] - 287s 5ms/step - loss: 0.0465 - acc: 0.9859 - val_loss: 0.0732 - val_acc: 0.9782
Epoch 5/5
60000/60000 [==============================] - 284s 5ms/step - loss: 0.0328 - acc: 0.9901 - val_loss: 0.0650 - val_acc: 0.9800
Test accuracy 1: 0.9221
Test accuracy 2: 0.0982


In [49]:
score1 = model2.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model2.evaluate(data[1]["test"], y_test, verbose=0)
print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])

# Save trained model
model2.save('model2.h5')

Test accuracy 1: 0.2351
Test accuracy 2: 0.98


In [50]:
model3 = load_model('model2.h5')

model3.fit(data[2]["train"], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(data[2]["test"], y_test))
score1 = model3.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model3.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model3.evaluate(data[2]["test"], y_test, verbose=0)
print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])

model3.save('model3.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 275s 5ms/step - loss: 0.2718 - acc: 0.9290 - val_loss: 0.1007 - val_acc: 0.9682
Epoch 2/5
60000/60000 [==============================] - 269s 4ms/step - loss: 0.0797 - acc: 0.9753 - val_loss: 0.1050 - val_acc: 0.9653
Epoch 3/5
60000/60000 [==============================] - 286s 5ms/step - loss: 0.0501 - acc: 0.9844 - val_loss: 0.0678 - val_acc: 0.9784
Epoch 4/5
60000/60000 [==============================] - 250s 4ms/step - loss: 0.0334 - acc: 0.9903 - val_loss: 0.0767 - val_acc: 0.9759
Epoch 5/5
60000/60000 [==============================] - 248s 4ms/step - loss: 0.0220 - acc: 0.9939 - val_loss: 0.0705 - val_acc: 0.9782
Test accuracy 1: 0.1178
Test accuracy 2: 0.3358
Test accuracy 3: 0.9782


In [51]:
# Reload model
model4 = load_model('model3.h5')

# Continue training
model4.fit(data[3]["train"], y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(data[3]["test"], y_test))
score1 = model4.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model4.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model4.evaluate(data[2]["test"], y_test, verbose=0)
score4 = model4.evaluate(data[3]["test"], y_test, verbose=0)

print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])
print('Test accuracy 4:', score4[1])

# Save trained model
model4.save('model4.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 105s 2ms/step - loss: 0.2233 - acc: 0.9437 - val_loss: 0.0884 - val_acc: 0.9731
Epoch 2/5
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0638 - acc: 0.9806 - val_loss: 0.0610 - val_acc: 0.9780
Epoch 3/5
60000/60000 [==============================] - 139s 2ms/step - loss: 0.0395 - acc: 0.9874 - val_loss: 0.0647 - val_acc: 0.9793
Epoch 4/5
60000/60000 [==============================] - 110s 2ms/step - loss: 0.0249 - acc: 0.9926 - val_loss: 0.0709 - val_acc: 0.9766
Epoch 5/5
60000/60000 [==============================] - 97s 2ms/step - loss: 0.0163 - acc: 0.9955 - val_loss: 0.0575 - val_acc: 0.9816
Test accuracy 1: 0.1631
Test accuracy 2: 0.1919
Test accuracy 3: 0.4445
Test accuracy 4: 0.9816


In [52]:
model5 = load_model('model4.h5')

# Continue training
model5.fit(data[4]["train"], y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(data[4]["test"], y_test))
score1 = model5.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model5.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model5.evaluate(data[2]["test"], y_test, verbose=0)
score4 = model5.evaluate(data[3]["test"], y_test, verbose=0)
score5 = model5.evaluate(data[4]["test"], y_test, verbose=0)

print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])
print('Test accuracy 4:', score4[1])
print('Test accuracy 5:', score5[1])


# Save trained model
model5.save('model5.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 107s 2ms/step - loss: 0.7884 - acc: 0.7661 - val_loss: 0.2741 - val_acc: 0.9085
Epoch 2/5
60000/60000 [==============================] - 98s 2ms/step - loss: 0.2635 - acc: 0.9169 - val_loss: 0.1934 - val_acc: 0.9413
Epoch 3/5
60000/60000 [==============================] - 102s 2ms/step - loss: 0.1764 - acc: 0.9447 - val_loss: 0.1446 - val_acc: 0.9525
Epoch 4/5
60000/60000 [==============================] - 104s 2ms/step - loss: 0.1328 - acc: 0.9583 - val_loss: 0.1363 - val_acc: 0.9575
Epoch 5/5
60000/60000 [==============================] - 104s 2ms/step - loss: 0.1231 - acc: 0.9608 - val_loss: 0.2707 - val_acc: 0.9143
Test accuracy 1: 0.109
Test accuracy 2: 0.1684
Test accuracy 3: 0.4884
Test accuracy 4: 0.9745
Test accuracy 5: 0.9143


In [53]:
model6 = load_model('model5.h5')

# Continue training
model6.fit(data[5]["train"], y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(data[5]["test"], y_test))
score1 = model6.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model6.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model6.evaluate(data[2]["test"], y_test, verbose=0)
score4 = model6.evaluate(data[3]["test"], y_test, verbose=0)
score5 = model6.evaluate(data[4]["test"], y_test, verbose=0)
score6 = model6.evaluate(data[5]["test"], y_test, verbose=0)

print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])
print('Test accuracy 4:', score4[1])
print('Test accuracy 5:', score5[1])
print('Test accuracy 6:', score6[1])

# Save trained model
model6.save('model6.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 183s 3ms/step - loss: 0.1640 - acc: 0.9540 - val_loss: 0.0891 - val_acc: 0.9708
Epoch 2/5
60000/60000 [==============================] - 176s 3ms/step - loss: 0.0518 - acc: 0.9837 - val_loss: 0.0648 - val_acc: 0.9788
Epoch 3/5
60000/60000 [==============================] - 124s 2ms/step - loss: 0.0313 - acc: 0.9902 - val_loss: 0.0555 - val_acc: 0.9837
Epoch 4/5
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0188 - acc: 0.9943 - val_loss: 0.0537 - val_acc: 0.9837
Epoch 5/5
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0109 - acc: 0.9970 - val_loss: 0.0842 - val_acc: 0.9760
Test accuracy 1: 0.0946
Test accuracy 2: 0.1595
Test accuracy 3: 0.5223
Test accuracy 4: 0.4856
Test accuracy 5: 0.908
Test accuracy 6: 0.976


In [54]:
model7 = load_model('model6.h5')

# Continue training
model7.fit(data[6]["train"], y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(data[6]["test"], y_test))
score1 = model7.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model7.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model7.evaluate(data[2]["test"], y_test, verbose=0)
score4 = model7.evaluate(data[3]["test"], y_test, verbose=0)
score5 = model7.evaluate(data[4]["test"], y_test, verbose=0)
score6 = model7.evaluate(data[5]["test"], y_test, verbose=0)
score7 = model7.evaluate(data[6]["test"], y_test, verbose=0)

print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])
print('Test accuracy 4:', score4[1])
print('Test accuracy 5:', score5[1])
print('Test accuracy 6:', score6[1])
print('Test accuracy 7:', score7[1])

# Save trained model
model7.save('model7.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 99s 2ms/step - loss: 0.3105 - acc: 0.9293 - val_loss: 0.1095 - val_acc: 0.9644
Epoch 2/5
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0739 - acc: 0.9770 - val_loss: 0.1338 - val_acc: 0.9572
Epoch 3/5
60000/60000 [==============================] - 99s 2ms/step - loss: 0.0422 - acc: 0.9868 - val_loss: 0.0789 - val_acc: 0.9769
Epoch 4/5
60000/60000 [==============================] - 100s 2ms/step - loss: 0.0256 - acc: 0.9927 - val_loss: 0.0571 - val_acc: 0.9836
Epoch 5/5
60000/60000 [==============================] - 109s 2ms/step - loss: 0.0142 - acc: 0.9961 - val_loss: 0.0618 - val_acc: 0.9826
Test accuracy 1: 0.0641
Test accuracy 2: 0.1241
Test accuracy 3: 0.5182
Test accuracy 4: 0.4298
Test accuracy 5: 0.8903
Test accuracy 6: 0.9086
Test accuracy 7: 0.9826


In [55]:
model8 = load_model('model7.h5')

# Continue training
model8.fit(data[7]["train"], y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(data[7]["test"], y_test))
score1 = model8.evaluate(data[0]["test"], y_test, verbose=0)
score2 = model8.evaluate(data[1]["test"], y_test, verbose=0)
score3 = model8.evaluate(data[2]["test"], y_test, verbose=0)
score4 = model8.evaluate(data[3]["test"], y_test, verbose=0)
score5 = model8.evaluate(data[4]["test"], y_test, verbose=0)
score6 = model8.evaluate(data[5]["test"], y_test, verbose=0)
score7 = model8.evaluate(data[6]["test"], y_test, verbose=0)
score8 = model8.evaluate(data[7]["test"], y_test, verbose=0)


print('Test accuracy 1:', score1[1])
print('Test accuracy 2:', score2[1])
print('Test accuracy 3:', score3[1])
print('Test accuracy 4:', score4[1])
print('Test accuracy 5:', score5[1])
print('Test accuracy 6:', score6[1])
print('Test accuracy 7:', score7[1])
print('Test accuracy 7:', score8[1])

# Save trained model
model8.save('model8.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 108s 2ms/step - loss: 0.4957 - acc: 0.8769 - val_loss: 0.2993 - val_acc: 0.9036
Epoch 2/5
60000/60000 [==============================] - 114s 2ms/step - loss: 0.1750 - acc: 0.9463 - val_loss: 0.4134 - val_acc: 0.8785
Epoch 3/5
60000/60000 [==============================] - 111s 2ms/step - loss: 0.1254 - acc: 0.9615 - val_loss: 0.2048 - val_acc: 0.9253
Epoch 4/5
60000/60000 [==============================] - 107s 2ms/step - loss: 0.1225 - acc: 0.9655 - val_loss: 0.0849 - val_acc: 0.9729
Epoch 5/5
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0813 - acc: 0.9746 - val_loss: 0.0824 - val_acc: 0.9732
Test accuracy 1: 0.1685
Test accuracy 2: 0.1415
Test accuracy 3: 0.4592
Test accuracy 4: 0.37
Test accuracy 5: 0.7466
Test accuracy 6: 0.7098
Test accuracy 7: 0.8192
Test accuracy 7: 0.9732


In [ ]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional layer 2 and pooling layer
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 2 with flattening
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    pool2_flat=tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense layer with dropout 
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    # Generate predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Caluclate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
# Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Logging predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, 
    every_n_iter=50)

# Our application logic will be added here

In [ ]:
def main(unused_argv):
    # Training on original
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":train_original},
        y=train_labels,
        batch_size=1000,
        num_epochs=None,
        shuffle=True)
    
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluation on original
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_original},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    
    eval_results=mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
  tf.app.run()